In [40]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
df = pd.read_csv("C:\\D\\datasets\\bank.csv", sep=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [42]:
df.shape

(4521, 17)

In [43]:
df.duplicated().sum()

0

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


In [45]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,4521.0,41.170095,10.576211,19.0,33.0,39.0,49.0,87.0
balance,4521.0,1422.657819,3009.638142,-3313.0,69.0,444.0,1480.0,71188.0
day,4521.0,15.915284,8.247667,1.0,9.0,16.0,21.0,31.0
duration,4521.0,263.961292,259.856633,4.0,104.0,185.0,329.0,3025.0
campaign,4521.0,2.793630,3.109807,1.0,1.0,2.0,3.0,50.0
pdays,4521.0,39.766645,100.121124,-1.0,-1.0,-1.0,-1.0,871.0
previous,4521.0,0.542579,1.693562,0.0,0.0,0.0,0.0,25.0


In [49]:
df.drop(['balance', 'pdays'], axis=1)

,age,job,marital,education,default,housing,loan,contact,day,month,duration,campaign,previous,poutcome,y
0,30,unemployed,married,primary,no,no,no,cellular,19,oct,79,1,0,NaN,no
1,33,services,married,secondary,no,yes,yes,cellular,11,may,220,1,4,failure,no
2,35,management,single,tertiary,no,yes,no,cellular,16,apr,185,1,1,failure,no
3,30,management,married,tertiary,no,yes,yes,NaN,3,jun,199,4,0,NaN,no
4,59,blue-collar,married,secondary,no,yes,no,NaN,5,may,226,1,0,NaN,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,yes,no,cellular,30,jul,329,5,0,NaN,no
4517,57,self-employed,married,tertiary,yes,yes,yes,NaN,9,may,153,1,0,NaN,no
4518,57,technician,married,secondary,no,no,no,cellular,19,aug,151,11,0,NaN,no
4519,28,blue-collar,married,secondary,no,no,no,cellular,6,feb,129,4,3,other,no


In [48]:
df = df.replace('unknown', np.nan)
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,NaN,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,NaN,3,jun,199,4,-1,0,NaN,no
4,59,blue-collar,married,secondary,no,0,yes,no,NaN,5,may,226,1,-1,0,NaN,no


In [50]:
df['month'] = pd.to_datetime(df['month'], format='%b')
df['month'] = df['month'].dt.month

In [51]:
df.isna().sum()

age             0
job            38
marital         0
education     187
default         0
balance         0
housing         0
loan            0
contact      1324
day             0
month           0
duration        0
campaign        0
pdays           0
previous        0
poutcome     3705
y               0
dtype: int64

In [52]:
df = df.drop(['contact', 'poutcome', 'day'], axis=1)
df.head()

,age,job,marital,education,default,balance,housing,loan,month,duration,campaign,pdays,previous,y
0,30,unemployed,married,primary,no,1787,no,no,10,79,1,-1,0,no
1,33,services,married,secondary,no,4789,yes,yes,5,220,1,339,4,no
2,35,management,single,tertiary,no,1350,yes,no,4,185,1,330,1,no
3,30,management,married,tertiary,no,1476,yes,yes,6,199,4,-1,0,no
4,59,blue-collar,married,secondary,no,0,yes,no,5,226,1,-1,0,no


In [53]:
cols_with_nan = [col for col in df if df[col].isnull().sum() > 0]
cols_with_nan

['job', 'education']

In [ ]:
data = df.copy()
nan_axis = np.where(data['job'].isna())[0]
data['is_nan'] = 0
data.loc[nan_axis, 'is_nan'] = 1

for col in data.select_dtypes(include='object'):
    data[col] = pd.factorize(data[col])[0]

train_data = data[data['is_nan'] == 0]
test_data = data[data['is_nan'] == 1]

X_train = train_data.drop(['job', 'is_nan'], axis=1)
y_train = train_data['job']
X_test = test_data.drop(['job', 'is_nan'], axis=1)

model = LGBMClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
df.loc[nan_axis, 'job'] = y_pred

In [ ]:
data = df.copy()
nan_axis = np.where(data['education'].isna())[0]
data['is_nan'] = 0
data.loc[nan_axis, 'is_nan'] = 1

for col in data.select_dtypes(include='object'):
    data[col] = pd.factorize(data[col])[0]

train_data = data[data['is_nan'] == 0]
test_data = data[data['is_nan'] == 1]

X_train = train_data.drop(['education', 'is_nan'], axis=1)
y_train = train_data['education']
X_test = test_data.drop(['education', 'is_nan'], axis=1)

model = LGBMClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
df.loc[nan_axis, 'education'] = y_pred

In [56]:
df.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
month        0
duration     0
campaign     0
pdays        0
previous     0
y            0
dtype: int64